**The submission consists of 3 different files:**


1.   Image training file (VGG)
2.   Noisy text training file (MLP)
3.   Final file (merging VGG and MLP with features (gender,basecolour,usage, and season) and a final training)

**Approach:**

The approach to tackle this ecommerce trainign was as follows: 


First I trained the model using images only and got a 27 probability vector for each data set. 
Then I trained another model using the noisy text description and again got a 27 probability vector for each data set. 
At the final file, I trained the model using the information gaiend by the previous two models mentioned and I concatenated those two 27 size vectors with each data set features after one hot encoding (vector of size 5+46+4+7=62). I considered this new vector of size 116 as the entry to the final model. 

**Run Guide Details:**


To run this code first you need to run two other models named **Final Text_Classifying Kaggle** and **Final VGG Kaggle** in that code there is a save command that save all the information of the training models into the directory. After this step you need to save those files to a file into your google drive (drag and dropping works too). After running the "drive.mount" command you have to give access to your drive by copying a link and there you have the information of previous trained models needed to run this final trianing model.


**Few Points:**

I left the intermediate running commands as comment so it helps you understand the code.

### **Libraries**

In [1]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import numpy as np
from tensorflow.keras import optimizers
import tensorflow.keras as keras
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Data Loading**

In [2]:
!pip install kaggle
!pip install --upgrade kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c uw-cs480-fall20
!ls
import zipfile
zip_ref = zipfile.ZipFile('uw-cs480-fall20.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=1fca84b936fa6dad7fe52fea31a968e3e6cb0a33fbdbf87cf6b7ecf7b487ac55
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.9
    Uninstalling kaggle-1.5.9:
      Successfully uninstalled kaggle-1.5.9


Saving kaggle.json to kaggle.json
 88% 75.0M/85.6M [00:00<00:00, 86.5MB/s]
100% 85.6M/85.6M [00:00<00:00, 123MB/s] 
drive  kaggle.json  sample_data  uw-cs480-fall20.zip


In [3]:
import pandas as pd
train = pd.read_csv('/content/files/train.csv',dtype=str)
test = pd.read_csv('/content/files/test.csv',dtype=str)

def Addjpg(fn):
    return fn+".jpg"

train["id"]=train["id"].apply(Addjpg)
test["id"]=test["id"].apply(Addjpg)   

train_folder = '/content/files/suffled-images/shuffled-images'
test_folder = '/content/files/suffled-images/shuffled-images'

Converting features to Numbers

In [4]:
def feature2Num (train,cat):
  le = preprocessing.LabelEncoder()
  ax=train[cat]
  ax2=pd.Categorical(ax)
  le.fit(ax2)
  ax3 = le.transform(ax2)
  return pd.get_dummies(ax3).to_numpy()

gender = feature2Num(train,"gender")

# print(gender.shape)

In [5]:
gender = feature2Num(train,"gender")
baseColour = feature2Num(train,"baseColour")
season = feature2Num(train,"season")
usage = feature2Num(train,"usage")
feature_train = np.concatenate((gender, baseColour, season, usage), axis=1)
feature_train_lable = feature2Num(train,"category")
print(feature_train.shape)

gender = feature2Num(test,"gender")
baseColour = feature2Num(test,"baseColour")
season = feature2Num(test,"season")
usage = feature2Num(test,"usage")
feature_test = np.concatenate((gender, baseColour, season, usage), axis=1)

from sklearn.model_selection import train_test_split

(21627, 62)


### **Loading the VGG16 Model training information**

In [6]:
model2 = keras.models.load_model('/content/drive/MyDrive/my_model_92.h2')

VGG16 Data Loading

In [7]:
# Define data generator
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

X_train_img = train_gen.flow_from_dataframe(dataframe = train, 
directory = train_folder, x_col = 'id', 
y_col = 'category', subset="training", seed = 42,
batch_size = 100, shuffle = False, 
class_mode="categorical",target_size = (80, 60))

X_test_img = test_gen.flow_from_dataframe(dataframe = test, 
directory = test_folder, x_col = 'id', 
y_col = None,
batch_size = 100, shuffle = False, 
class_mode=None,target_size = (80, 60))

Found 21627 validated image filenames belonging to 27 classes.
Found 21628 validated image filenames.


Getting the probability output of VGG Model for both train and test



In [8]:
vgg_out_train = model2.predict_proba(X_train_img,verbose=1)
vgg_out_test = model2.predict_proba(X_test_img,verbose=1)

Instructions for updating:
Please use `model.predict()` instead.
217/217 [==============================] - 9s 42ms/step


### **Loading the Noisy Text Model training information**

In [9]:
model3 = keras.models.load_model('/content/drive/MyDrive/MLP_89.h5')


Noisy Text Data Loading

In [10]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences

Text_train = train['noisyTextDescription'].values
labels = train['category'].values
text_test = test['noisyTextDescription'].values

encoder = LabelBinarizer()
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(Text_train)
tokenizer.fit_on_texts(text_test)

Text_train = tokenizer.texts_to_sequences(Text_train)
text_test = tokenizer.texts_to_sequences(text_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = 30

Text_train = pad_sequences(Text_train, padding='post', maxlen=maxlen)
text_test = pad_sequences(text_test, padding='post', maxlen=maxlen)

labels = encoder.fit_transform(labels) 




Getting the probability output of Noisy Text Model for both train and test


In [11]:
nlp_train = model3.predict_proba(Text_train)
nlp_test = model3.predict_proba(text_test)


In [12]:
# print(nlp_test.shape)
# print(sentences_test.shape)
# print(nlp_train.shape)
# print(sentences.shape)

In [13]:
X_train_conc=np.concatenate((vgg_out_train,feature_train,nlp_train), axis=1)
X_test_conc=np.concatenate((vgg_out_test,feature_test,nlp_test),axis=1)

# print(vgg_out_train.shape)
# print(feature_train.T.shape)
# print(vgg_out_test.shape)
# print(feature_test.T.shape)

# print(X_train_conc.shape)
# print(X_test_conc.shape)
# print(feature_train_lable.shape)


In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train_conc, feature_train_lable, test_size=0.05, random_state=0)
# y_train = pd.get_dummies(y_train).to_numpy()
# y_val = pd.get_dummies(y_val).to_numpy()
X_test = X_test_conc;
# print(X_train.shape)
# print(X_val.shape)
# print(y_train.shape)

### **Main Model Training**

In [20]:
model = Sequential()
model.add(Dense(1024, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.5))
model.add(Dense(27))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=100, verbose=1)
# evaluate the keras model
_, accuracy = model.evaluate(X_val, y_val, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))


Epoch 1/20
206/206 [==============================] - 1s 4ms/step - loss: 0.5352 - accuracy: 0.8719 - val_loss: 0.0739 - val_accuracy: 0.9871
Epoch 2/20
206/206 [==============================] - 1s 3ms/step - loss: 0.0895 - accuracy: 0.9816 - val_loss: 0.0510 - val_accuracy: 0.9898
Epoch 3/20
206/206 [==============================] - 1s 3ms/step - loss: 0.0645 - accuracy: 0.9868 - val_loss: 0.0507 - val_accuracy: 0.9898
Epoch 4/20
206/206 [==============================] - 1s 3ms/step - loss: 0.0544 - accuracy: 0.9882 - val_loss: 0.0514 - val_accuracy: 0.9889
Epoch 5/20
206/206 [==============================] - 1s 3ms/step - loss: 0.0458 - accuracy: 0.9898 - val_loss: 0.0522 - val_accuracy: 0.9880
Epoch 6/20
206/206 [==============================] - 1s 3ms/step - loss: 0.0428 - accuracy: 0.9912 - val_loss: 0.0427 - val_accuracy: 0.9908
Epoch 7/20
206/206 [==============================] - 1s 3ms/step - loss: 0.0410 - accuracy: 0.9912 - val_loss: 0.0453 - val_accuracy: 0.9908
Epoch 

Exporting to Excel

In [36]:
pred=model.predict_generator(X_test,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

labels = (X_train_img.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=X_test_img.filenames
j=0
for i in filenames:
  filenames[j] = filenames[j].replace('.jpg','')
  j=j+1
# print(filenames)
results_vgg=pd.DataFrame({"id":filenames,"category":predictions})
results_vgg.to_csv("results_rmsp_vgg_hot_tex.csv",index=False)

676/676 [==============================] - 1s 1ms/step
